In [10]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
import cv2
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [12]:
from utils import get_binary_mask_from_image, \
                get_leftx_rightx_base,\
                get_left_right_lane_xy,\
                get_lane_line_coefficients,\
                get_lane_line_position,\
                get_lane_pts_for_fillPoly,\
                get_lane_rectangle_image,\
                weighted_img,\
                get_lane_curvature,\
                print_curvature_info

In [13]:
test_images_path = glob.glob('./test_images/*_dst.jpg')
test_images_path

['./test_images/straight_lines1_dst.jpg',
 './test_images/test2_dst.jpg',
 './test_images/test6_dst.jpg',
 './test_images/test3_dst.jpg',
 './test_images/test1_dst.jpg',
 './test_images/test4_dst.jpg',
 './test_images/straight_lines2_dst.jpg',
 './test_images/test5_dst.jpg']

In [14]:
with open('experimental/cfg.pickle', 'rb') as f:
    cfg = pickle.load(f)
    
with open('experimental/data.pickle', 'rb') as f:
    data = pickle.load(f)
    
with open('experimental/perspective_mtx.pickle', 'rb') as f:
    perspective_mtx = pickle.load(f)

In [18]:
cap = cv2.VideoCapture('./project_video.mp4')

fourcc = cv2.VideoWriter_fourcc(*'XVID')
frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
size = (width, height)
out = cv2.VideoWriter('project_output.mp4', fourcc, fps, size)

for ith in tqdm(range(frames)):
    
    # frame : BGR-scale image
    ret, frame = cap.read()
    
    if ret == True:
        frame = dst = cv2.undistort(frame, data['mtx'], data['dist'], None, data['newcameramtx'])
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        binary = get_binary_mask_from_image(frame, cfg)
        binary_warped = cv2.warpPerspective(binary, perspective_mtx['M'], size)
        leftx_base, rightx_base = get_leftx_rightx_base(binary_warped)
        leftx, lefty, rightx, righty = get_left_right_lane_xy(binary_warped, 
                                                              leftx_base, 
                                                              rightx_base)
        left_fit, right_fit = get_lane_line_coefficients(leftx, lefty, rightx, righty)
        left_fitx, right_fitx, ploty = get_lane_line_position(size[1], left_fit, right_fit)
        lane_pts = get_lane_pts_for_fillPoly(left_fitx, right_fitx, ploty)
        img_lane_warped = get_lane_rectangle_image(binary_warped, lane_pts)
        img_lane = cv2.warpPerspective(img_lane_warped, perspective_mtx['MInv'], size)
        
        out_img = weighted_img(img_lane, frame_rgb)
        left_curvature, right_curvature = get_lane_curvature(left_fit, right_fit, ploty)
        window_img = print_curvature_info(out_img, left_curvature, right_curvature)
        
        out.write(window_img)
        
    else:
        break
        
cap.release()
out.release()

100%|██████████| 1/1 [00:00<00:00,  5.04it/s]


In [20]:
def create_video(in_file, out_file, data, cfg, perspective_mtx):
    cap = cv2.VideoCapture(in_file)

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    size = (width, height)
    out = cv2.VideoWriter(out_file, fourcc, fps, size)

    for ith in tqdm(range(frames)):

        # frame : BGR-scale image
        ret, frame = cap.read()

        if ret == True:
            frame = dst = cv2.undistort(frame, data['mtx'], data['dist'], None, data['newcameramtx'])
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            binary = get_binary_mask_from_image(frame, cfg)
            binary_warped = cv2.warpPerspective(binary, perspective_mtx['M'], size)
            leftx_base, rightx_base = get_leftx_rightx_base(binary_warped)
            leftx, lefty, rightx, righty = get_left_right_lane_xy(binary_warped, 
                                                                  leftx_base, 
                                                                  rightx_base)
            left_fit, right_fit = get_lane_line_coefficients(leftx, lefty, rightx, righty)
            left_fitx, right_fitx, ploty = get_lane_line_position(size[1], left_fit, right_fit)
            lane_pts = get_lane_pts_for_fillPoly(left_fitx, right_fitx, ploty)
            img_lane_warped = get_lane_rectangle_image(binary_warped, lane_pts)
            img_lane = cv2.warpPerspective(img_lane_warped, perspective_mtx['MInv'], size)

            out_img = weighted_img(img_lane, frame_rgb)
            left_curvature, right_curvature = get_lane_curvature(left_fit, right_fit, ploty)
            window_img = print_curvature_info(out_img, left_curvature, right_curvature)

            out.write(window_img)

        else:
            break

    cap.release()
    out.release()

In [ ]:
create_video('./project_video.mp4', './project_output.mp4',
             data, cfg, perspective_mtx)

 83%|████████▎ | 1041/1260 [02:42<00:38,  5.70it/s]

In [ ]:
create_video('./challenge_video.mp4', './challenge_output.mp4',
             data, cfg, perspective_mtx)

In [ ]:
create_video('./harder_challenge_video.mp4', './harder_challenge_output.mp4',
             data, cfg, perspective_mtx)